In [47]:
# importing libs
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, Dense, GaussianNoise,Lambda
from keras.models import Model
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam,SGD
from keras import backend as K

In [48]:
# for reproducing reslut
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(3)

In [49]:
# defining parameters
M = 4
k = np.log2(M)
k = int(k)
R = 1
n_channel = 2
print ('M:',M,'k:',k,'n:',n_channel)

M: 4 k: 2 n: 2


In [50]:
#generating data of size N
N = 8000
label = np.random.randint(M,size=N)

In [51]:
# creating one hot encoded vectors
data = []
for i in label:
    temp = np.zeros(M)
    temp[i] = 1
    data.append(temp)

In [52]:
data = np.array(data)
print (data.shape)

(8000, 4)


In [53]:
temp_check = [17,23,45,67,89,96,72,250,350]
for i in temp_check:
    print(label[i],data[i])

1 [ 0.  1.  0.  0.]
0 [ 1.  0.  0.  0.]
1 [ 0.  1.  0.  0.]
3 [ 0.  0.  0.  1.]
1 [ 0.  1.  0.  0.]
3 [ 0.  0.  0.  1.]
0 [ 1.  0.  0.  0.]
3 [ 0.  0.  0.  1.]
0 [ 1.  0.  0.  0.]


In [54]:
def antirectifier(x):
    y = x/K.l2_normalize(x,axis=0)
    return 
def antirectifier_output_shape(input_shape):
    return input_shape

In [201]:
input_signal = Input(shape=(M,))
encoded = Dense(M, activation='relu')(input_signal)
encoded1 = Dense(n_channel, activation='linear')(encoded)
encoded2 = Lambda(lambda x: x/tf.norm(x))(encoded1)

EbNo_train = 5.01187 #  coverted 7 db of EbNo
encoded3 = GaussianNoise(np.sqrt(1/(2*R*EbNo_train)))(encoded2)

decoded = Dense(M, activation='relu')(encoded3)
decoded1 = Dense(M, activation='softmax')(decoded)
autoencoder = Model(input_signal, decoded1)
adam = Adam(lr=0.01)
autoencoder.compile(optimizer=adam, loss='categorical_crossentropy')

In [202]:
print (autoencoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        (None, 4)                 0         
_________________________________________________________________
dense_107 (Dense)            (None, 4)                 20        
_________________________________________________________________
dense_108 (Dense)            (None, 2)                 10        
_________________________________________________________________
lambda_28 (Lambda)           (None, 2)                 0         
_________________________________________________________________
gaussian_noise_27 (GaussianN (None, 2)                 0         
_________________________________________________________________
dense_109 (Dense)            (None, 4)                 12        
_________________________________________________________________
dense_110 (Dense)            (None, 4)                 20        
Total para

In [203]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [205]:
autoencoder.fit(data, data,
                epochs=100,
                batch_size=256)

Epoch 1/100
8000/8000 [==============================] - 0s - loss: 1.3700     
Epoch 2/100
8000/8000 [==============================] - 0s - loss: 1.3680     
Epoch 3/100
8000/8000 [==============================] - 0s - loss: 1.3699     
Epoch 4/100
8000/8000 [==============================] - 0s - loss: 1.3684     
Epoch 5/100
8000/8000 [==============================] - 0s - loss: 1.3650     
Epoch 6/100
8000/8000 [==============================] - 0s - loss: 1.3651     
Epoch 7/100
8000/8000 [==============================] - 0s - loss: 1.3650     
Epoch 8/100
8000/8000 [==============================] - 0s - loss: 1.3671     
Epoch 9/100
8000/8000 [==============================] - 0s - loss: 1.3661     
Epoch 10/100
8000/8000 [==============================] - 0s - loss: 1.3670     
Epoch 11/100
8000/8000 [==============================] - 0s - loss: 1.3687     
Epoch 12/100
8000/8000 [==============================] - 0s - loss: 1.3678     
Epoch 13/100
8000/8000 [=============

In [63]:
from keras.models import load_model
#autoencoder.save('2_2_symbol_autoencoder_v_best.model')

In [64]:
#autoencoder_loaded = load_model('4_7_symbol_autoencoder_v_best.model')

In [196]:
encoder = Model(input_signal, encoded2)

In [197]:
encoded_input = Input(shape=(n_channel,))

deco = autoencoder.layers[-2](encoded_input)
deco = autoencoder.layers[-1](deco)
# create the decoder model
decoder = Model(encoded_input, deco)

In [198]:
N = 5000
test_label = np.random.randint(M,size=N)
test_data = []

for i in test_label:
    temp = np.zeros(M)
    temp[i] = 1
    test_data.append(temp)
    
test_data = np.array(test_data)

In [199]:
temp_test = 6
print (test_data[temp_test][test_label[temp_test]],test_label[temp_test])

1.0 1


In [200]:
print (encoder.predict(np.expand_dims([0,0,0,1],axis=0)))
print (encoder.predict(np.expand_dims([0,0,1,0],axis=0)))
print (encoder.predict(np.expand_dims([0,1,0,0],axis=0)))
print (encoder.predict(np.expand_dims([1,0,0,0],axis=0)))

[[ 0.99308366  0.11740891]]
[[ 0.99988073  0.01544695]]
[[ 0.89591837  0.44421872]]
[[-0.99092066 -0.13444807]]


In [116]:
def frange(x, y, jump):
  while x < y:
    yield x
    x += jump

In [117]:
EbNodB_range = list(frange(-2,10.5,0.5))
ber = [None]*len(EbNodB_range)
for n in range(0,len(EbNodB_range)):
    EbNo=10.0**(EbNodB_range[n]/10.0)
    noise_std = np.sqrt(1/(2*R*EbNo))
    noise_mean = 0
    no_errors = 0
    nn = N
    noise = noise_std * np.random.randn(nn,n_channel)
    encoded_signal = encoder.predict(test_data) 
    final_signal = encoded_signal + noise
    pred_final_signal =  decoder.predict(final_signal)
    pred_output = np.argmax(pred_final_signal,axis=1)
    no_errors = (pred_output != test_label)
    no_errors =  no_errors.astype(int).sum()
    ber[n] = no_errors / nn 
    print ('SNR:',EbNodB_range[n],'BER:',ber[n])

SNR: -2 BER: 0.6306
SNR: -1.5 BER: 0.6118
SNR: -1.0 BER: 0.6062
SNR: -0.5 BER: 0.6048
SNR: 0.0 BER: 0.5958
SNR: 0.5 BER: 0.5762
SNR: 1.0 BER: 0.5802
SNR: 1.5 BER: 0.558
SNR: 2.0 BER: 0.5442
SNR: 2.5 BER: 0.534
SNR: 3.0 BER: 0.5206
SNR: 3.5 BER: 0.5064
SNR: 4.0 BER: 0.5008
SNR: 4.5 BER: 0.4768
SNR: 5.0 BER: 0.462
SNR: 5.5 BER: 0.4412
SNR: 6.0 BER: 0.4248
SNR: 6.5 BER: 0.4138
SNR: 7.0 BER: 0.3848
SNR: 7.5 BER: 0.369
SNR: 8.0 BER: 0.3244
SNR: 8.5 BER: 0.311
SNR: 9.0 BER: 0.3032
SNR: 9.5 BER: 0.2724
SNR: 10.0 BER: 0.2558


In [118]:
import matplotlib.pyplot as plt
from scipy import interpolate
plt.plot(EbNodB_range, ber, 'bo',label='Autoencoder(2,2)')
#tck = interpolate.splrep(EbNodB_range, ber, s=0)
#xnew = np.arange(-2,8.8, 0.6)
#ynew = interpolate.splev(xnew, tck, der=0)
#plt.plot(xnew,ynew,'y')
#plt.plot(list(EbNodB_range), ber_theory, 'ro-',label='BPSK BER')
plt.yscale('log')
plt.xlabel('SNR Range')
plt.ylabel('Block Error Rate')
plt.grid()
plt.legend(loc='upper right',ncol = 1)

In [ ]:
plt.savefig('AutoEncoder_2_2_BER_matplotlib')
plt.show()